## Week list

In [1]:
import os
import random
import yaml
import math
from collections import defaultdict
from datetime import date

import ipywidgets as widgets
from IPython.display import display

from src.Settings import RECIPE_DIR
from src.Recipe import list_recipes, get_ingredient_unit, update_selection, Recipe

recipe_list = list_recipes()
recipe_names = list(recipe_list.keys())

today = date.today()
recipe_names_with_selection = []
for name, recipe in recipe_list.items():
    recipe_obj =  Recipe.load(recipe["filename"])
    score = recipe_obj.compute_seasonal_score()
    if recipe.get("selection"):
        recipe_names_with_selection.append(f"{(today - recipe.get('selection')).days} - {name} -   {score}")
    else:
        recipe_names_with_selection.append(f"Never - {name} -   {score}")
recipe_names_with_selection = sorted(recipe_names_with_selection)
        

this_week_list = widgets.VBox(
    [
        widgets.BoundedIntText(value="7", min=0, max=10, description="Number of recipes:"),
        widgets.HBox([widgets.Dropdown(options=recipe_names_with_selection,
                                description='Select',
                                disabled=False), widgets.Button(disabled=False, button_style='', tooltip='Add', icon='plus')]),
        widgets.Label(value="This week's list: "),
        widgets.VBox([]),
        widgets.Button(
            disabled=False,
            button_style='',
            tooltip='Generate',
            icon='play',
        ),
        widgets.Button(
            disabled=False,
            button_style='',
            tooltip='Shopping list',
            icon='store',
        ),
    ]
)

def add_button(b=None):
    print(this_week_list.children[1].children[0].value)
    add_recipe(this_week_list.children[1].children[0].value.split(" - ")[1])

def change_button(b=None):
    selection = random.randint(0, len(recipe_names)-1)
    this_week_list.children[3].children[int(b.tooltip)].children[-1].value = f"{recipe_names[selection]}"

def remove_button(b=None):
    print(b.tooltip)
    this_week_list.children[3].children[int(b.tooltip)].children=()

def add_recipe(name):
    existing_names  = [child.children[-1].value for child in this_week_list.children[3].children]
    if name in existing_names:
        print("Warning: recipe already included. Skipping.")
        return False
    i = len(this_week_list.children[3].children)
    this_week_list.children[3].children += (widgets.HBox(
        [
            widgets.Button(
                disabled=False,
                button_style='',
                tooltip=str(i),
                icon='pen',
            ),
            widgets.Button(
                disabled=False,
                button_style='',
                tooltip=str(i),
                icon='minus',
            ),
            widgets.Label(value=f"{name}"),
        ]),
    )
    this_week_list.children[3].children[-1].children[0].on_click(change_button)
    this_week_list.children[3].children[-1].children[1].on_click(remove_button)
    return True
    

def generate_recipes(b):
    if this_week_list.children[3].children:
        this_week_list.children[3].children = ()
    for i in range(this_week_list.children[0].value):
        selection = random.randint(0, len(recipe_names)-1)
        while not add_recipe(recipe_names[selection]):
              selection = random.randint(0, len(recipe_names)-1)

def get_general_ingredient_name(name, ingredient_dict):
    for ingredient_name, ingredient_obj in ingredient_dict.items():
        possible_names = [format_ingredient_name(ingredient_name)]
        for ingredient_alt in ingredient_obj.get("alternatives", []):
            possible_names.append(format_ingredient_name(ingredient_alt))
        if format_ingredient_name(name) in possible_names:
            return ingredient_name
    
    raise Exception(f"Ingredient '{name}' not known. Please add it. {possible_names}")

def create_shopping_list(b=None):
    
    # retrieve existing ingredients
    ingredient_dict = None
    with open(os.path.join(os.getcwd(),"rsrc", "ingredients.yaml"), 'r') as ingf:
        ingredient_dict = yaml.load(ingf, Loader=yaml.FullLoader)
    
    # Process all recipes selected, extract ingredients and add then to the shopping list
    shopping_list = {}
    weekly_list = []
    for recipe_widget in this_week_list.children[3].children:
        recipe = Recipe.load(recipe_list[recipe_widget.children[-1].value]["filename"], recipe_list[recipe_widget.children[-1].value]["id"])
        for recipe_ingredient in recipe.ingredients:
            name = get_general_ingredient_name(recipe_ingredient["name"], ingredient_dict)
            if recipe_ingredient.get("description"):
                name += " (" + recipe_ingredient.get("description") + ")"
            
            unit, value = get_ingredient_unit(recipe_ingredient)
            if unit is None or value is None:
                raise Exception(f"Ingredient {ingredient} with None value (from {recipe_widget.children[-1].value})")
            
            if name in shopping_list.keys():
                if unit != shopping_list[name]["unit"]:
                    raise Exception(f"Ingredients with different units. {name} ({unit}/{shopping_list[name]['unit']})" )
                shopping_list[name]["value"] += value
            else:
                shopping_list[name] = {"unit": unit, "value": value}
        update_selection(recipe.name)
        weekly_list.append(recipe.name)
    
    # saving the shopping list and this week's meals
    with open("shopping_list.txt", "w") as sl:
        for name, content in shopping_list.items():
            if content["unit"] == "x":
                sl.write(f"- {name}: x {math.ceil(content['value'])}\n")
            else:
                sl.write(f"- {name}: {content['value']}{content['unit']}\n")
                
    with open("weekly_list.yaml", "w") as wlf:
        yaml.dump(weekly_list, wlf, default_flow_style=False)
    print("Shopping list created successfully!")
    
this_week_list.children[1].children[-1].on_click(add_button)        
this_week_list.children[-2].on_click(generate_recipes)
this_week_list.children[-1].on_click(create_shopping_list)
display(this_week_list)

with open(os.path.join(os.getcwd(), "weekly_list.yaml"), 'r') as wlf:
    week_recipes = yaml.load(wlf, Loader=yaml.FullLoader)
    for recipe in week_recipes:
        add_recipe(recipe)

In [3]:
display(widgets.HTML(
    value="<a href='http://192.168.1.133:8080/voila/render/git/PyRecipe/show_recipes.ipynb'><b>Show recipes</b></a>"
))
display(widgets.HTML(
    value="<a href='http://192.168.1.133:8080/voila/render/git/PyRecipe/create_recipe.ipynb'><b>Create or edit recipe</b></a>"
))

HTML(value="<a href='http://192.168.1.133:8080/voila/render/git/PyRecipe/show_recipes.ipynb'><b>Show recipes</…

HTML(value="<a href='http://192.168.1.133:8080/voila/render/git/PyRecipe/create_recipe.ipynb'><b>Create or edi…

In [9]:
with open(os.path.join(os.getcwd(), "shopping_list.txt"), 'r') as wlf:
    shopping_list = wlf.read()
    print(shopping_list)

- potato: 2250.0g
- carrot: x 5
- onion: x 4
- rice: 500g
- curry (japanese curry cube): x 1
- quesadillas: x 8
- pepper: x 4
- haricot (noir): 200.0g
- cheddar: 400.0g
- chili (fort): x 1
- chili (doux): x 2
- coriander: x 1
- lime: x 1
- tomato (conserve): x 2
- aubergine: x 5
- coulis de tomate: 0.2L
- feta: 200g
- thym: x 1
- oil (huile d'olive ): x 2
- apples: x 4
- butter: 250.0g
- garlic: x 11
- courgette: x 2
- tomato: x 4
- oil (olive): 6.0tbs
- bouquet garnis: x 1
- fish (Morue salée (cod fish)): 900.0g
- milk: 1.0L
- pain de mie (tranches): x 16
- mozzarella (boules): x 2
- pesto: x 1
- pepper (en boite): x 2

